# Socrata API Basics 
## Socrata Open Data API (SODA) Tutorial Using NYC Open Data
Author: Mark Bauer

Table of Contents
=================

   1. Introduction
   2. Sodapy
       - 2.1 Using Sodapy
   3. Importing Libraries
   4. Sodapy Socrata Class
   5. Socrata APIs
       - 5.1 Socrata API
       - 5.2 Discovery API
       - 5.3 Metadata API

# 1. Introduction  
This notebook demonstrates how to interact with the [Socrata Open Data API (SODA)](https://dev.socrata.com/) using the [sodapy](https://github.com/xmunoz/sodapy) Python client and explores different methods for retrieving data from Socrata-based open data portals. Not only will we learn how to fetch data, but we'll also explore Socrata's available metadata using the [Metadata](https://dev.socrata.com/docs/other/metadata#?route=overview) and [Discovery](https://dev.socrata.com/docs/other/discovery#?route=overview) APIs. For this tutorial, we will use data from NYC Open Data as an example.

While the main focus is on understanding how to interact with the Socrata API, we'll use the sodapy library throughout the examples, as it provides a straightforward interface for working with Socrata. Both the raw Socrata API URLs and sodapy code snippets will be provided to give you flexibility in how you approach your data retrieval.

# 2. Sodapy

## 2.1 Using Sodapy

Sodapy is a python client for the Socrata API. In order use sodapy, a source domain (i.e. the Socrata Open Data source you are trying to connect to) needs to be passed to the Socrata class. Additionally, if a user wants to query a specific dataset on Socrata Open Data, then the dataset identifier (i.e. the dataset id on the given source domain) needs to be passed as well. Below, we identify NYC Open Data's source domain: `data.cityofnewyork.us` and the dataset identifier for the NYC 311 dataset: `erm2-nwe9`. The screenshot below displays where we retrieve this information.

This is my preferred method for extracting data from NYC Open Data. However, please note that the Sodapy project is now archived on GitHub.

![nyc-311-api-docs](images/nyc-311-api-docs.png)  

**Source**: https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9

We will be focus on three sodapy methods:
- `.get()`
Read data from the requested resource.

-  `.datasets()`
Returns the list of datasets associated with a particular domain.

- `.get_metadata()`
Retrieve the metadata for a particular dataset.

For a comprehensive list of all available APIs provided by Socrata, check out the [Other APIs](https://dev.socrata.com/docs/other/) page in the documentation.

# 3. Importing Libraries

In [1]:
# importing libraries
import pandas as pd
from sodapy import Socrata
import requests

In [2]:
# documention for installing watermark: https://github.com/rasbt/watermark
# performed for reproducibility
%reload_ext watermark
%watermark -u -t -d -v -p pandas,sodapy

Last updated: 2024-11-30 19:49:05

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

pandas: 1.5.1
sodapy: 2.2.0



# 4. Sodapy Socrata Class 

Note:  
`WARNING:root:Requests made without an app_token will be subject to strict throttling limits.`  

To avoid these limits, it's recommended to use an app token when making API requests.

In [3]:
## implementation in sodapy
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Socrata(): The main class that interacts with the SODA API.

# The required arguments are:
#     domain: the domain you wish you to access
#     app_token: your Socrata application token
# Simple requests are possible without an app_token, though these
# requests will be rate-limited.

client = Socrata(
    socrata_domain,
    app_token=None,
    timeout=100
)

print(client)

In [4]:
# print information about the Socrata object
print(f'type: {type(client)}')

type: <class 'sodapy.socrata.Socrata'>


In [5]:
# print attributes of object
for key, value in client.__dict__.items():
    print(f'{key}: {value}')

domain: data.cityofnewyork.us
session: <requests.sessions.Session object at 0x10cd9c0d0>
uri_prefix: https://
timeout: 100


# 5. Socrata APIs

##  5.1 Socrata Open Data API
[Socrata's Open Data API](https://dev.socrata.com/docs/endpoints).

From [the docs](https://dev.socrata.com/docs/endpoints):
>The “endpoint” of a SODA API is simply a unique URL that represents an object or collection of objects. Every Socrata dataset, and even every individual data record, has its own endpoint. The endpoint is what you’ll point your HTTP client at to interact with data resources.
>
>All resources are accessed through a common base path of /resource/ along with their dataset identifier.

Sodapy `.get()` method: Read data from the requested resource. Options for content_type are JSON,
CSV, and XML.

This method performs a get request on these type of URLs: https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=5.

### Using Socrata API URL

In [6]:
# limit 311 dataset to 5 rows
url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=5'
df = pd.read_json(url)

# preview data
df.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,address_type,...,latitude,longitude,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_7mpf_4k6g,location_type
0,63231133,2024-11-29T04:00:33.000,DOT,Department of Transportation,Street Condition,Pothole,11213.0,ALBANY AVENUE,CROWN STREET,INTERSECTION,...,40.665687,-73.939760,"{'latitude': '40.66568740802441', 'longitude':...",17615,17,2,48,44,44,NaN
1,63233902,2024-11-29T01:51:28.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.892051,-73.860009,"{'latitude': '40.89205062779512', 'longitude':...",11275,29,5,2,30,30,Residential Building/House
2,63231561,2024-11-29T01:51:03.000,DPR,Department of Parks and Recreation,Illegal Tree Damage,Bicycle Chained to Tree,NaN,NaN,NaN,NaN,...,40.831950,-73.929394,"{'latitude': '40.83195011687482', 'longitude':...",10930,50,5,35,27,27,Street
3,63232135,2024-11-29T01:50:50.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.892051,-73.860009,"{'latitude': '40.89205062779512', 'longitude':...",11275,29,5,2,30,30,Residential Building/House
4,63233043,2024-11-29T01:50:14.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.734143,-73.774759,"{'latitude': '40.73414322554409', 'longitude':...",14508,25,3,16,65,65,Residential Building/House


### Using Sodapy

In [7]:
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# NYC 311 dataset identifier
socrata_dataset_identifier = 'erm2-nwe9'

# initialize client
client = Socrata(
    socrata_domain,
    app_token=None,
    timeout=1000
)

# get data for the 311 dataset, limit to 5 rows
# notice the limit parameter
data = client.get(socrata_dataset_identifier, limit=5)

# identify type of object returned
print(type(data))

<class 'list'>


In [8]:
# preview first element in list
data[0]

{'unique_key': '63231133',
 'created_date': '2024-11-29T04:00:33.000',
 'agency': 'DOT',
 'agency_name': 'Department of Transportation',
 'complaint_type': 'Street Condition',
 'descriptor': 'Pothole',
 'incident_zip': '11213',
 'intersection_street_1': 'ALBANY AVENUE',
 'intersection_street_2': 'CROWN STREET',
 'address_type': 'INTERSECTION',
 'city': 'BROOKLYN',
 'facility_type': 'N/A',
 'status': 'Open',
 'resolution_description': 'The Department of Transportation referred this complaint to the appropriate Maintenance Unit for repair.',
 'resolution_action_updated_date': '2024-11-29T04:00:33.000',
 'community_board': '09 BROOKLYN',
 'borough': 'BROOKLYN',
 'x_coordinate_state_plane': '1000962',
 'y_coordinate_state_plane': '181810',
 'open_data_channel_type': 'UNKNOWN',
 'park_facility_name': 'Unspecified',
 'park_borough': 'BROOKLYN',
 'latitude': '40.66568740802441',
 'longitude': '-73.93975997355022',
 'location': {'latitude': '40.66568740802441',
  'longitude': '-73.939759973550

In [9]:
# convert list to a df
df = pd.DataFrame(data)

# sanity check
print(df.shape)
df.head()

(5, 32)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,address_type,...,latitude,longitude,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_7mpf_4k6g,location_type
0,63231133,2024-11-29T04:00:33.000,DOT,Department of Transportation,Street Condition,Pothole,11213,ALBANY AVENUE,CROWN STREET,INTERSECTION,...,40.66568740802441,-73.93975997355022,"{'latitude': '40.66568740802441', 'longitude':...",17615,17,2,48,44,44,NaN
1,63233902,2024-11-29T01:51:28.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.89205062779512,-73.86000893554395,"{'latitude': '40.89205062779512', 'longitude':...",11275,29,5,2,30,30,Residential Building/House
2,63231561,2024-11-29T01:51:03.000,DPR,Department of Parks and Recreation,Illegal Tree Damage,Bicycle Chained to Tree,NaN,NaN,NaN,NaN,...,40.83195011687482,-73.92939377993235,"{'latitude': '40.83195011687482', 'longitude':...",10930,50,5,35,27,27,Street
3,63232135,2024-11-29T01:50:50.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.89205062779512,-73.86000893554395,"{'latitude': '40.89205062779512', 'longitude':...",11275,29,5,2,30,30,Residential Building/House
4,63233043,2024-11-29T01:50:14.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,40.73414322554409,-73.7747588426775,"{'latitude': '40.73414322554409', 'longitude':...",14508,25,3,16,65,65,Residential Building/House


Example with the QUERY parameter.

In [10]:
## SoQL implementation with sodapy
# SoQL query string below:
# retrieve all columns and limit our records to 100

query = """
    SELECT *
    LIMIT 100
"""

# returned as JSON from API / converted to Python list of dictionaries by sodapy
# notice the query parameter
results = client.get(socrata_dataset_identifier, query=query)

# convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# sanity check
print(f'shape of data: {results_df.shape}')
results_df.head()

shape of data: (100, 43)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,address_type,...,bridge_highway_direction,bridge_highway_segment,vehicle_type,incident_address,street_name,cross_street_1,cross_street_2,landmark,bbl,closed_date
0,63231133,2024-11-29T04:00:33.000,DOT,Department of Transportation,Street Condition,Pothole,11213,ALBANY AVENUE,CROWN STREET,INTERSECTION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63233902,2024-11-29T01:51:28.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63231561,2024-11-29T01:51:03.000,DPR,Department of Parks and Recreation,Illegal Tree Damage,Bicycle Chained to Tree,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63232135,2024-11-29T01:50:50.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63233043,2024-11-29T01:50:14.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5.2  Discovery API
Socrata's [Discovery API](https://dev.socrata.com/docs/other/discovery#?route=overview).

Sodapy `.datasets()` method: Returns the list of datasets associated with a particular domain.
WARNING: Large limits (>1000) will return megabytes of data,
which can be slow on low-bandwidth networks, and is also a lot of
data to hold in memory.

This method performs a get request on these type of URLs: https://data.cityofnewyork.us/api/catalog/v1.

### Using Socrata API URL

In [11]:
# discovery api
url = 'https://api.us.socrata.com/api/catalog/v1?search_context=data.ny.gov'

# Fetch the JSON data from the web
response = requests.get(url)

# Parse the JSON response
data_dict = response.json() 

# preview keys    
data_dict.keys()    

dict_keys(['results', 'resultSetSize', 'timings', 'warnings'])

In [12]:
# preview results key, first element
data_dict['results'][0]

{'resource': {'name': 'Lottery Cash 4 Life Winning Numbers: Beginning 2014',
  'id': 'kwxv-fwze',
  'resource_name': None,
  'parent_fxf': [],
  'description': 'Go to http://on.ny.gov/1xRIvPz on the New York Lottery website for past Cash 4 Life results and payouts.',
  'attribution': 'New York State Gaming Commission',
  'attribution_link': 'http://nylottery.ny.gov/wps/portal/Home/Lottery/home/your+lottery/drawing+results/drawingresults_cash4life',
  'contact_email': 'opendata@its.ny.gov',
  'type': 'dataset',
  'updatedAt': '2024-11-30T11:03:52.000Z',
  'createdAt': '2014-06-17T19:47:54.000Z',
  'metadata_updated_at': '2024-11-30T11:03:51.000Z',
  'data_updated_at': '2024-11-30T11:03:52.000Z',
  'page_views': {'page_views_last_week': 426,
   'page_views_last_month': 2664,
   'page_views_total': 5932662,
   'page_views_last_week_log': 8.73809225962049,
   'page_views_last_month_log': 11.379919817646538,
   'page_views_total_log': 22.500248403354632},
  'columns_name': ['Winning Numbers

In [13]:
# convert into df
df = pd.DataFrame(data_dict['results'])

# sanity check
print(df.shape)
df.head()

(100, 8)


,resource,classification,metadata,permalink,link,owner,creator,preview_image_url
0,{'name': 'Lottery Cash 4 Life Winning Numbers:...,"{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.ny.gov'},https://data.ny.gov/d/kwxv-fwze,https://data.ny.gov/Government-Finance/Lottery...,"{'id': 'xzik-pf59', 'user_type': 'interactive'...","{'id': 'xzik-pf59', 'user_type': 'interactive'...",NaN
1,"{'name': 'For Hire Vehicles (FHV) - Active', '...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/8wbx-tsch,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
2,"{'name': 'Civil Service List (Active)', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/vx8i-nprf,https://data.cityofnewyork.us/City-Government/...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
3,"{'name': 'DOB Job Application Filings', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/ic3t-wcy2,https://data.cityofnewyork.us/Housing-Developm...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
4,"{'name': 'Medicaid Enrolled Provider Listing',...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'health.data.ny.gov'},https://health.data.ny.gov/d/keti-qx5t,https://health.data.ny.gov/Health/Medicaid-Enr...,"{'id': 's9j2-nqmr', 'user_type': 'interactive'...","{'id': 's9j2-nqmr', 'user_type': 'interactive'...",NaN


This is not our final dataframe. Our information most likely is located in the resource column, but let's confirm.

Briefly review the other keys.

In [14]:
# first element in our results list, preview resources key
data_dict['results'][0]['resource']

{'name': 'Lottery Cash 4 Life Winning Numbers: Beginning 2014',
 'id': 'kwxv-fwze',
 'resource_name': None,
 'parent_fxf': [],
 'description': 'Go to http://on.ny.gov/1xRIvPz on the New York Lottery website for past Cash 4 Life results and payouts.',
 'attribution': 'New York State Gaming Commission',
 'attribution_link': 'http://nylottery.ny.gov/wps/portal/Home/Lottery/home/your+lottery/drawing+results/drawingresults_cash4life',
 'contact_email': 'opendata@its.ny.gov',
 'type': 'dataset',
 'updatedAt': '2024-11-30T11:03:52.000Z',
 'createdAt': '2014-06-17T19:47:54.000Z',
 'metadata_updated_at': '2024-11-30T11:03:51.000Z',
 'data_updated_at': '2024-11-30T11:03:52.000Z',
 'page_views': {'page_views_last_week': 426,
  'page_views_last_month': 2664,
  'page_views_total': 5932662,
  'page_views_last_week_log': 8.73809225962049,
  'page_views_last_month_log': 11.379919817646538,
  'page_views_total_log': 22.500248403354632},
 'columns_name': ['Winning Numbers', 'Cash Ball', 'Draw Date'],
 '

In [15]:
# first element in our results list, preview classification key
data_dict['results'][0]['classification']

{'categories': [],
 'tags': [],
 'domain_category': 'Government & Finance',
 'domain_tags': ['cash 4 life', 'new york lottery', 'results', 'winning'],
 'domain_metadata': [{'key': 'Common-Core_Publisher',
   'value': 'State of New York'},
  {'key': 'Common-Core_Contact-Name', 'value': 'Open Data NY'},
  {'key': 'Common-Core_Contact-Email', 'value': 'opendata@its.ny.gov'},
  {'key': 'Additional-Resources_See-Also',
   'value': 'http://www.gaming.ny.gov/'},
  {'key': 'Dataset-Summary_Dataset-Owner',
   'value': 'New York State Gaming Commission'},
  {'key': 'Dataset-Summary_Contact-Information',
   'value': 'Info@gaming.ny.gov'},
  {'key': 'Dataset-Summary_Granularity', 'value': 'By draw'},
  {'key': 'Dataset-Summary_Coverage', 'value': 'Statewide'},
  {'key': 'Dataset-Summary_Data-Frequency',
   'value': 'Daily beginning 7/1/19; twice weekly previously'},
  {'key': 'Dataset-Summary_Posting-Frequency', 'value': 'Daily'},
  {'key': 'Dataset-Summary_Organization', 'value': 'The New York Lo

In [16]:
# first element in our results list, preview metadata key
data_dict['results'][0]['metadata']

{'domain': 'data.ny.gov'}

In [17]:
# first element in our results list, preview permalink key
data_dict['results'][0]['permalink']

'https://data.ny.gov/d/kwxv-fwze'

In [18]:
# first element in our results list, preview link key
data_dict['results'][0]['link']

'https://data.ny.gov/Government-Finance/Lottery-Cash-4-Life-Winning-Numbers-Beginning-2014/kwxv-fwze'

In [19]:
# first element in our results list, preview owner key
data_dict['results'][0]['owner']

{'id': 'xzik-pf59', 'user_type': 'interactive', 'display_name': 'NY Open Data'}

In [20]:
# first element in our results list, preview creator key
data_dict['results'][0]['creator']

{'id': 'xzik-pf59', 'user_type': 'interactive', 'display_name': 'NY Open Data'}

In [21]:
# retrieve information in the resource key
# switch back to our dataframe object
df['resource']

0     {'name': 'Lottery Cash 4 Life Winning Numbers:...
1     {'name': 'For Hire Vehicles (FHV) - Active', '...
2     {'name': 'Civil Service List (Active)', 'id': ...
3     {'name': 'DOB Job Application Filings', 'id': ...
4     {'name': 'Medicaid Enrolled Provider Listing',...
                            ...                        
95    {'name': 'Lottery Pick 10 Winning Numbers: Beg...
96    {'name': 'Street Hail Livery (SHL) Permits', '...
97    {'name': 'NYPD Complaint Data Current (Year To...
98    {'name': 'Index Crimes by County and Agency: B...
99    {'name': 'TLC Approved LabCorp Patient Service...
Name: resource, Length: 100, dtype: object

In [22]:
# normalize JSON
df = pd.json_normalize(df['resource'])

# sanity check
print(df.shape)
df.head()

(100, 32)


,name,id,resource_name,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,...,locked,blob_mime_type,hide_from_data_json,publication_date,page_views.page_views_last_week,page_views.page_views_last_month,page_views.page_views_total,page_views.page_views_last_week_log,page_views.page_views_last_month_log,page_views.page_views_total_log
0,Lottery Cash 4 Life Winning Numbers: Beginning...,kwxv-fwze,None,[],Go to http://on.ny.gov/1xRIvPz on the New York...,New York State Gaming Commission,http://nylottery.ny.gov/wps/portal/Home/Lotter...,opendata@its.ny.gov,dataset,2024-11-30T11:03:52.000Z,...,False,None,False,2021-04-27T14:13:45.000Z,426,2664,5932662,8.738092,11.379920,22.500248
1,For Hire Vehicles (FHV) - Active,8wbx-tsch,None,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2024-11-30T20:05:48.000Z,...,False,None,False,2021-04-05T13:20:47.000Z,1,12,186,1.000000,3.700440,7.546894
2,Civil Service List (Active),vx8i-nprf,None,[],A Civil Service List consists of all candidate...,Department of Citywide Administrative Services...,None,None,dataset,2024-11-29T14:20:44.000Z,...,False,None,False,2024-01-12T16:15:05.000Z,0,0,0,NaN,NaN,NaN
3,DOB Job Application Filings,ic3t-wcy2,None,[],This dataset contains all job applications sub...,Department of Buildings (DOB),None,None,dataset,2024-11-30T21:06:12.000Z,...,False,None,False,2020-06-22T18:23:35.000Z,2,14,79,1.584963,3.906891,6.321928
4,Medicaid Enrolled Provider Listing,keti-qx5t,None,[],<b>Revalidation disclaimer</b>: The next anti...,New York State Department of Health,https://www.emedny.org/info/ProviderEnrollment...,None,dataset,2024-11-25T19:21:50.000Z,...,False,None,False,2020-12-28T16:03:15.000Z,0,0,0,NaN,NaN,NaN


In [23]:
# inspect available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   name                                  100 non-null    object 
 1   id                                    100 non-null    object 
 2   resource_name                         1 non-null      object 
 3   parent_fxf                            100 non-null    object 
 4   description                           100 non-null    object 
 5   attribution                           89 non-null     object 
 6   attribution_link                      51 non-null     object 
 7   contact_email                         20 non-null     object 
 8   type                                  100 non-null    object 
 9   updatedAt                             100 non-null    object 
 10  createdAt                             100 non-null    object 
 11  metadata_updated_at 

In [24]:
# preview columns and values
df.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,Lottery Cash 4 Life Winning Numbers: Beginning...,For Hire Vehicles (FHV) - Active,Civil Service List (Active),DOB Job Application Filings,Medicaid Enrolled Provider Listing,TLC New Driver Application Status,Lottery Daily Numbers/Win-4 Winning Numbers: B...,For Hire Vehicles (FHV) - Active Drivers,311 Service Requests from 2010 to Present,Civil Service List Certification,...,Farmers' Markets in New York State,Spring Farmers' Markets in New York State Map,"COVID-19 Daily Counts of Cases, Hospitalizatio...",Hospital Inpatient Cost Transparency: Beginnin...,Jobs NYC Postings,Lottery Pick 10 Winning Numbers: Beginning 1987,Street Hail Livery (SHL) Permits,NYPD Complaint Data Current (Year To Date),Index Crimes by County and Agency: Beginning 1990,TLC Approved LabCorp Patient Services Drug Tes...
id,kwxv-fwze,8wbx-tsch,vx8i-nprf,ic3t-wcy2,keti-qx5t,dpec-ucu7,hsys-3def,xjfq-wh2d,erm2-nwe9,a9md-ynri,...,qq4h-8p86,ezkt-j7iy,rc75-m7u3,7dtz-qxmr,kpav-sd4t,bycu-cw7c,yhuu-4pt3,5uac-w243,ca8h-8gjq,xzxm-9msq
resource_name,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
parent_fxf,[],[],[],[],[],[],[],[],[],[],...,[],"[qq4h-8p86, wseq-ucz9]",[],[],[],[],[],[],[],[pe54-wf39]
description,Go to http://on.ny.gov/1xRIvPz on the New York...,"<b>PLEASE NOTE:</b> This dataset, which includ...",A Civil Service List consists of all candidate...,This dataset contains all job applications sub...,<b>Revalidation disclaimer</b>: The next anti...,THIS DATASET IS UPDATED SEVERAL TIMES PER DAY....,Go to http://on.ny.gov/1Cx6zvs or http://on.ny...,"<b>PLEASE NOTE:</b> This dataset, which includ...",<b>NOTE:</b> The 311 dataset is currently show...,A List Certification includes the names of eli...,...,In the past decade the number of farmers' mark...,In the past decade the number of farmers' mark...,Daily count of NYC residents who tested positi...,This dataset contains information submitted by...,This dataset contains current job postings ava...,Go to http://on.ny.gov/1Cx6Zls on the New York...,"<b>Please Note:</b> This dataset, which includ...","This dataset includes all valid felony, misdem...",The Division of Criminal Justice Services (DCJ...,List of TLC approved LabCorp Drug Test Locatio...
attribution,New York State Gaming Commission,Taxi and Limousine Commission (TLC),Department of Citywide Administrative Services...,Department of Buildings (DOB),New York State Department of Health,Taxi and Limousine Commission (TLC),New York State Gaming Commission,Taxi and Limousine Commission (TLC),311,Department of Citywide Administrative Services...,...,New York State Department of Agriculture and M...,New York State Department of Agriculture and M...,Department of Health and Mental Hygiene (DOHMH),New York State Department of Health,Department of Citywide Administrative Services...,New York State Gaming Commission,Taxi and Limousine Commission (TLC),Police Department (NYPD),New York State Division of Criminal Justice Se...,Taxi and Limousine Commission (TLC)
attribution_link,http://nylottery.ny.gov/wps/portal/Home/Lotter...,None,None,None,https://www.emedny.org/info/ProviderEnrollment...,None,http://nylottery.ny.gov/wps/portal/Home/Lotter...,None,None,None,...,http://www.agriculture.ny.gov/AP/CommunityFarm...,http://www.agriculture.ny.gov/AP/CommunityFarm...,None,http://www.health.ny.gov/statistics/sparcs/,https://cityjobs.nyc.gov/,http://nylottery.ny.gov/wps/portal/Home/Lotter...,None,None,http://www.criminaljustice.ny.gov/crimnet/ojsa...,None
contact_email,opendata@its.ny.gov,None,None,None,None,None,opendata@its.ny.gov,None,None,None,...,opendata@its.ny.gov,opendata@its.ny.gov,None,None,None,opendata@its.ny.gov,None,None,None,None
type,dataset,dataset,dataset,dataset,dataset,dataset,dataset,dataset,dataset,dataset,...,dataset,map,dataset,dataset,dataset,dataset,dataset,dataset,dataset,map
updatedAt,2024-11-30T11:03:52.000Z,2024-11-30T20:05:48.000Z

### Using Sodapy

In [25]:
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# initialize client
client = Socrata(
    socrata_domain,
    app_token=None,
    timeout=100
)

datasets = client.datasets()

print(f'object type: {type(datasets)}')
print(f'Number of datasets on NYC Open Data: {len(datasets):,}.')

object type: <class 'list'>
Number of datasets on NYC Open Data: 3,235.


In [26]:
# review type about first dataset
print(type(datasets[0]))

<class 'dict'>


In [27]:
# review information about keys first dataset
datasets[0].keys()

dict_keys(['resource', 'classification', 'metadata', 'permalink', 'link', 'owner', 'creator'])

In [28]:
# review information about resource key
resources = datasets[0]['resource'].items()

for key, value in resources:
    print(f'{key}: {value}\n')      

name: For Hire Vehicles (FHV) - Active

id: 8wbx-tsch

resource_name: None

parent_fxf: []

description: <b>PLEASE NOTE:</b> This dataset, which includes all TLC licensed for-hire vehicles which are in good standing and able to drive, is updated every day in the evening between 4-7pm. Please check the 'Last Update Date' field to make sure the list has updated successfully. 'Last Update Date'  should show either today or yesterday's date, depending on the time of day. If the list is outdated, please download the most recent list from the link below. 
http://www1.nyc.gov/assets/tlc/downloads/datasets/tlc_for_hire_vehicle_active_and_inactive.csv

TLC authorized For-Hire vehicles that are active. This list is accurate to the date and time represented in the Last Date Updated and Last Time Updated fields. For inquiries about the contents of this dataset, please email licensinginquiries@tlc.nyc.gov.

attribution: Taxi and Limousine Commission (TLC)

attribution_link: None

contact_email: Non

Retrieve information about 311 dataset.

In [29]:
## Retrieve information about 311 dataset
# NYC 311 dataset identifier
socrata_dataset_identifier = 'erm2-nwe9'

# get the list of datasets once
datasets = client.datasets()

# loop through the datasets to find the one with the matching identifier
for idx, dataset in enumerate(datasets):
    if dataset['resource']['id'] == socrata_dataset_identifier:
        print('We found the NYC 311 dataset!')
        print(f'Index is: {idx}')
        
        dataset_index = idx
        break

We found the NYC 311 dataset!
Index is: 5


In [30]:
# normalize JSON
df = pd.json_normalize(datasets[dataset_index]['resource'])

# sanity check
print(df.shape)
df.head()

(1, 32)


,name,id,resource_name,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,...,locked,blob_mime_type,hide_from_data_json,publication_date,page_views.page_views_last_week,page_views.page_views_last_month,page_views.page_views_total,page_views.page_views_last_week_log,page_views.page_views_last_month_log,page_views.page_views_total_log
0,311 Service Requests from 2010 to Present,erm2-nwe9,None,[],<b>NOTE:</b> The 311 dataset is currently show...,311,None,None,dataset,2024-11-30T02:32:46.000Z,...,False,None,False,2023-12-01T06:51:46.000Z,2548,12514,907673,11.315716,13.611371,19.791815


In [31]:
# preview available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   name                                  1 non-null      object 
 1   id                                    1 non-null      object 
 2   resource_name                         0 non-null      object 
 3   parent_fxf                            1 non-null      object 
 4   description                           1 non-null      object 
 5   attribution                           1 non-null      object 
 6   attribution_link                      0 non-null      object 
 7   contact_email                         0 non-null      object 
 8   type                                  1 non-null      object 
 9   updatedAt                             1 non-null      object 
 10  createdAt                             1 non-null      object 
 11  metadata_updated_at    

In [32]:
# preview columns and values
df.T

,0
name,311 Service Requests from 2010 to Present
id,erm2-nwe9
resource_name,None
parent_fxf,[]
description,<b>NOTE:</b> The 311 dataset is currently show...
attribution,311
attribution_link,None
contact_email,None
type,dataset
updatedAt,2024-11-30T02:32:46.000Z


## 5.3  Metadata API
Socrata's [Metadata API](https://dev.socrata.com/docs/other/metadata#?route=overview).

Sodapy `.get_metadata()` method: Retrieve the metadata for a particular dataset.

### Using Socrata API URL

In [33]:
# example of the Metadata API with 311 dataset
# https://dev.socrata.com/docs/other/metadata#?route=overview

# all datasets on NYC Open Data, and expand nested columns
url = 'https://data.cityofnewyork.us/api/views/'
df = pd.read_json(url)

# preview data
df.head()

,id,name,assetType,averageRating,category,createdAt,description,displayType,downloadCount,hideFromCatalog,...,blobId,blobMimeType,rowIdentifierColumnId,queryString,ratings,indexUpdatedAt,childViews,iconUrl,previewImageId,disabledFeatureFlags
0,6xyb-j5pk,NYC Address Points (Map),map,0,City Government,1732652029,Address points were developed to supplement th...,visualization_canvas_map,0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,uf93-f8nk,NYC Address Points,dataset,0,City Government,1732641922,Address points were developed to supplement th...,table,13,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b7aj-ck5a,NYC Greenhouse Gas Emissions Municipal Inventory,dataset,0,Environment,1732116009,The Inventory of New York City Greenhouse Gas ...,table,37,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3g6p-4u5s,Building Footprints (Map),map,0,City Government,1731516162,Shapefile of footprint outlines of buildings i...,visualization_canvas_map,1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,u9wf-3gbt,Building Footprints (P Layer),dataset,0,City Government,1731513726,Shapefile of footprint outlines of buildings i...,table,20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# preview available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        3235 non-null   object 
 1   name                      3235 non-null   object 
 2   assetType                 3235 non-null   object 
 3   averageRating             3235 non-null   int64  
 4   category                  3135 non-null   object 
 5   createdAt                 3235 non-null   int64  
 6   description               3170 non-null   object 
 7   displayType               3235 non-null   object 
 8   downloadCount             3235 non-null   int64  
 9   hideFromCatalog           3235 non-null   bool   
 10  hideFromDataJson          3235 non-null   bool   
 11  locked                    3235 non-null   bool   
 12  modifyingViewUid          271 non-null    object 
 13  newBackend                3235 non-null   bool   
 14  numberOf

In [35]:
# Discovery API, 311 dataset
# instead of expanding nested columns, let's keep only top-level columns
url = 'https://data.cityofnewyork.us/api/views/erm2-nwe9'

# fetch the JSON data from the web
response = requests.get(url)

# parse the JSON response
data_dict = response.json()  

# preview keys    
data_dict.keys()    

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'locked', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowClass', 'rowIdentifierColumnId', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [36]:
# convert to df
df = pd.DataFrame([data_dict])

df.head()

,id,name,assetType,attribution,averageRating,category,createdAt,description,displayType,downloadCount,...,clientContext,columns,grants,metadata,owner,query,rights,tableAuthor,tags,flags
0,erm2-nwe9,311 Service Requests from 2010 to Present,dataset,311,0,Social Services,1318225937,<b>NOTE:</b> The 311 dataset is currently show...,table,444226,...,"{'clientContextVariables': [], 'inheritedVaria...","[{'id': 585605889, 'name': 'Unique Key', 'data...","[{'inherited': False, 'type': 'viewer', 'flags...","{'rdfSubject': '0', 'rdfClass': '', 'jsonQuery...","{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...","{'orderBys': [{'ascending': False, 'expression...",[read],"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...","[311, 311 service requests, city government, s...","[default, ownerMayBeContacted, restorable, res..."


In [37]:
# view columns
# notice the decrease in number of columns compared to the previous example
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1 non-null      object
 1   name                      1 non-null      object
 2   assetType                 1 non-null      object
 3   attribution               1 non-null      object
 4   averageRating             1 non-null      int64 
 5   category                  1 non-null      object
 6   createdAt                 1 non-null      int64 
 7   description               1 non-null      object
 8   displayType               1 non-null      object
 9   downloadCount             1 non-null      int64 
 10  hideFromCatalog           1 non-null      bool  
 11  hideFromDataJson          1 non-null      bool  
 12  locked                    1 non-null      bool  
 13  newBackend                1 non-null      bool  
 14  numberOfComments          1 no

In [38]:
# example of the Metadata API with 311 dataset
# https://dev.socrata.com/docs/other/metadata#?route=overview

# all datasets on NYC Open Data, notice the metadata in the URL
url = 'https://data.cityofnewyork.us/api/views/metadata/v1/'
df = pd.read_json(url)

# preview data
df.head()

,id,name,attribution,attributionLink,category,createdAt,dataUpdatedAt,dataUri,description,domain,...,hideFromCatalog,hideFromDataJson,license,metadataUpdatedAt,provenance,updatedAt,webUri,approvals,customFields,tags
0,6xyb-j5pk,NYC Address Points (Map),Office of Technology and Innovation (OTI),None,City Government,2024-11-26T20:13:49+0000,2024-11-26T18:02:43+0000,https://data.cityofnewyork.us/resource/6xyb-j5pk,Address points were developed to supplement th...,data.cityofnewyork.us,...,False,False,None,2024-11-26T20:14:56+0000,OFFICIAL,2024-11-26T20:14:56+0000,https://data.cityofnewyork.us/d/6xyb-j5pk,"[{'reviewedAt': 1732652072, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...",None
1,uf93-f8nk,NYC Address Points,Office of Technology and Innovation (OTI),None,City Government,2024-11-26T17:25:22+0000,2024-11-26T18:02:43+0000,https://data.cityofnewyork.us/resource/uf93-f8nk,Address points were developed to supplement th...,data.cityofnewyork.us,...,False,False,None,2024-11-26T20:13:07+0000,OFFICIAL,2024-11-26T20:13:07+0000,https://data.cityofnewyork.us/d/uf93-f8nk,"[{'reviewedAt': 1732651869, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...",[address point]
2,b7aj-ck5a,NYC Greenhouse Gas Emissions Municipal Inventory,Mayor's Office of Climate and Environmental Ju...,https://climate.cityofnewyork.us/initiatives/n...,Environment,2024-11-20T15:20:09+0000,2024-11-20T15:48:44+0000,https://data.cityofnewyork.us/resource/b7aj-ck5a,The Inventory of New York City Greenhouse Gas ...,data.cityofnewyork.us,...,False,False,None,2024-11-20T15:49:27+0000,OFFICIAL,2024-11-20T15:49:49+0000,https://data.cityofnewyork.us/d/b7aj-ck5a,"[{'reviewedAt': 1732117789, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[greenhouse gas emissions, greenhouse, fuel, e..."
3,3g6p-4u5s,Building Footprints (Map),Office of Technology and Innovation (OTI),None,City Government,2024-11-13T16:42:42+0000,2024-11-26T17:51:19+0000,https://data.cityofnewyork.us/resource/3g6p-4u5s,Shapefile of footprint outlines of buildings i...,data.cityofnewyork.us,...,False,False,None,2024-11-13T16:47:48+0000,OFFICIAL,2024-11-13T16:47:48+0000,https://data.cityofnewyork.us/d/3g6p-4u5s,"[{'reviewedAt': 1731516188, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[footprints, buildings]"
4,u9wf-3gbt,Building Footprints (P Layer),Office of Technology and Innovation (OTI),None,City Government,2024-11-13T16:02:06+0000,2024-11-26T17:33:50+0000,https://data.cityofnewyork.us/resource/u9wf-3gbt,Shapefile of footprint outlines of buildings i...,data.cityofnewyork.us,...,False,False,None,2024-11-26T17:32:07+0000,OFFICIAL,2024-11-26T17:32:07+0000,https://data.cityofnewyork.us/d/u9wf-3gbt,"[{'reviewedAt': 1731516092, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[building, footprint, footprints]"


In [39]:
# preview available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3235 non-null   object 
 1   name               3235 non-null   object 
 2   attribution        3168 non-null   object 
 3   attributionLink    449 non-null    object 
 4   category           3135 non-null   object 
 5   createdAt          3235 non-null   object 
 6   dataUpdatedAt      3058 non-null   object 
 7   dataUri            3235 non-null   object 
 8   description        3170 non-null   object 
 9   domain             3235 non-null   object 
 10  externalId         0 non-null      float64
 11  hideFromCatalog    3235 non-null   bool   
 12  hideFromDataJson   3235 non-null   bool   
 13  license            79 non-null     object 
 14  metadataUpdatedAt  3235 non-null   object 
 15  provenance         3235 non-null   object 
 16  updatedAt          3235 

In [40]:
# Metadata API for 311 dataset
url = 'https://data.cityofnewyork.us/api/views/metadata/v1/erm2-nwe9/'

# Fetch the JSON data from the web
response = requests.get(url)

# Parse the JSON response
data_dict = response.json()

# convert to a df
df = pd.DataFrame([data_dict])

# preview data
df.head()

,id,name,attribution,attributionLink,category,createdAt,dataUpdatedAt,dataUri,description,domain,...,hideFromCatalog,hideFromDataJson,license,metadataUpdatedAt,provenance,updatedAt,webUri,approvals,customFields,tags
0,erm2-nwe9,311 Service Requests from 2010 to Present,311,None,Social Services,2011-10-10T05:52:17+0000,2024-11-30T02:32:46+0000,https://data.cityofnewyork.us/resource/erm2-nwe9,<b>NOTE:</b> The 311 dataset is currently show...,data.cityofnewyork.us,...,False,False,None,2024-05-28T20:32:16+0000,OFFICIAL,2024-05-28T20:32:16+0000,https://data.cityofnewyork.us/d/erm2-nwe9,"[{'reviewedAt': 1524193398, 'reviewedAutomatic...","{'Update': {'Automation': 'Yes', 'Date Made Pu...","[311, 311 service requests, city government, s..."


In [41]:
# preview available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1 non-null      object
 1   name               1 non-null      object
 2   attribution        1 non-null      object
 3   attributionLink    0 non-null      object
 4   category           1 non-null      object
 5   createdAt          1 non-null      object
 6   dataUpdatedAt      1 non-null      object
 7   dataUri            1 non-null      object
 8   description        1 non-null      object
 9   domain             1 non-null      object
 10  externalId         0 non-null      object
 11  hideFromCatalog    1 non-null      bool  
 12  hideFromDataJson   1 non-null      bool  
 13  license            0 non-null      object
 14  metadataUpdatedAt  1 non-null      object
 15  provenance         1 non-null      object
 16  updatedAt          1 non-null      object
 17  w

### Using Sodapy

In [42]:
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# dataset id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# initialize client
client = Socrata(
    socrata_domain,
    app_token=None,
    timeout=100
)

# similar to: 'https://data.cityofnewyork.us/api/views/metadata/v1/'
metadata = client.get_metadata(socrata_dataset_identifier)

metadata.keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'locked', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowClass', 'rowIdentifierColumnId', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [43]:
# convert to df
df = pd.DataFrame([metadata])

# preview data
df.head()

,id,name,assetType,attribution,averageRating,category,createdAt,description,displayType,downloadCount,...,clientContext,columns,grants,metadata,owner,query,rights,tableAuthor,tags,flags
0,erm2-nwe9,311 Service Requests from 2010 to Present,dataset,311,0,Social Services,1318225937,<b>NOTE:</b> The 311 dataset is currently show...,table,444226,...,"{'clientContextVariables': [], 'inheritedVaria...","[{'id': 585605889, 'name': 'Unique Key', 'data...","[{'inherited': False, 'type': 'viewer', 'flags...","{'rdfSubject': '0', 'rdfClass': '', 'jsonQuery...","{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...","{'orderBys': [{'ascending': False, 'expression...",[read],"{'id': '5fuc-pqz2', 'displayName': 'NYC OpenDa...","[311, 311 service requests, city government, s...","[default, ownerMayBeContacted, restorable, res..."


In [44]:
# preview available columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1 non-null      object
 1   name                      1 non-null      object
 2   assetType                 1 non-null      object
 3   attribution               1 non-null      object
 4   averageRating             1 non-null      int64 
 5   category                  1 non-null      object
 6   createdAt                 1 non-null      int64 
 7   description               1 non-null      object
 8   displayType               1 non-null      object
 9   downloadCount             1 non-null      int64 
 10  hideFromCatalog           1 non-null      bool  
 11  hideFromDataJson          1 non-null      bool  
 12  locked                    1 non-null      bool  
 13  newBackend                1 non-null      bool  
 14  numberOfComments          1 no

In [45]:
# close connection
client.close()